**OpenViSUS: read from a remote dataset**

To execute this jupyter notebook have to install: OpenViSUS, numpy, matplotlib, ipyvolume

pip3 install numpy matplotlib ipyvolume

You can do it with pip3 (which uses python3):
pip3 install OpenVisus

If the installation of OpenVisus via pip does not work you have to build it and install it yourself (see OpenViSUS README), or add your build directory to the PYTHONPATH, or in your program add it to the sys path before the import:
sys.path.append('/somepath_to_your/OpenVisus/build')


In [ ]:
import os,sys
import math
import matplotlib.pyplot as plt
import numpy
import ipywidgets 

def ShowData(data):
    fig = plt.figure(figsize = (70,20))
    ax = fig.add_subplot(1,1,1)
    ax.imshow(data, origin='lower')
    plt.show()

In [ ]:
sys.path.append(r"D:\projects\OpenVisus\build\RelWithDebInfo")
from OpenVisus import *
DbModule.attach()

In [ ]:
db=PyDataset("http://molniya.sci.utah.edu/mod_visus?dataset=agricolture_time")
print(db.getDatasetBody().toString())

In [ ]:
# do not query full resolution but up to 2 ^20 samples in logic space

logic_box=db.getLogicBox(x=[0.0,1.0],y=[0.0,1.0])

def ReadData(time=db.getDefaultTime(),max_resolution=20):
    return next(db.read(logic_box=logic_box,time=time, max_resolution=max_resolution))

# if float range means alpha,beta
data=ReadData()
ShowData(data)

In [ ]:
def onTimeChanged(time):
    ShowData(ReadData(time=time))

ipywidgets.interact(
    onTimeChanged, 
    time=ipywidgets.widgets.IntSlider(value=0,min=0,max=4,step=1))

In [ ]:
data = ReadData(time=1)-ReadData(time=0)
ShowData(data)

In [ ]:
from OpenVisus.PyImage import * 
    
def rgb2gray(rgb):
    r, g, b = rgb[:,:,0], rgb[:,:,1], rgb[:,:,2]
    gray = 0.2989 * r + 0.5870 * g + 0.1140 * b
    return gray    
    
def onTimesChanged(time0,time1):
    ShowData(
        rgb2gray(ReadData(time=time1))-
        rgb2gray(ReadData(time=time0))
    )
    
ipywidgets.interact(onTimesChanged, 
    time0=ipywidgets.widgets.IntSlider(value=0,min=0, max=6, step=1), 
    time1=ipywidgets.widgets.IntSlider(value=1,min=0, max=6, step=1))
